In [ ]:
#Importamos la librerías para la lectura de nuestro dataset en formato parquet
import pyarrow as pa
import pyarrow.parquet as pq

In [ ]:
#Instanciamos en un dataframe
df = pd.read_parquet('/content/drive/MyDrive/Colab Notebooks/data.parquet')

In [ ]:
#Revisamos el número de id´s únicos con los que contamos antes de tomar un fracción del dataset
print(df['id'].nunique())

22998


In [ ]:
#Tomamos una muestra al 25 porciento (más adelante en el documento comprobe que este era el volumen de datos ideal)
data = df.sample(frac=.25, random_state=42)

In [ ]:
#Compruebo que todos los id´s se encuentren en la muestra
print(data['id'].nunique())

22998


In [ ]:
#Instalacion de la librería surprise para realizar el modelo
!pip install surprise

In [ ]:
#Mandamos a llamar las funciones necesarias de la librería
from surprise import Reader, Dataset, accuracy, SVD
from surprise.model_selection import cross_validate, train_test_split, GridSearchCV


In [ ]:
# Creamos el objeto Reader con las escalas del Score
reader = Reader(rating_scale=(0.5, 5))

# Carmos los datos en una estructura de Surprise
dataa = Dataset.load_from_df(data[['userId', 'id', 'score']], reader)

In [ ]:
#Realizamos la división de los datos de entrenamiento y de prueba
trainset, testset = train_test_split(dataa, test_size=0.2)

In [ ]:
#Definimos la grilla de hiperparámetros a explorar
param_grid = {'n_factors': [50, 100, 150], 'n_epochs': [10, 20, 30], 'lr_all': [0.002, 0.005, 0.01], 'reg_all': [0.02, 0.04, 0.06]}


In [ ]:
#Defino la métrica de evaluación
scoring = {'RMSE': 'neg_root_mean_squared_error'}

In [ ]:
#Inicializamos el objeto GridSearchCV
gs = GridSearchCV(SVD, param_grid, measures=['rmse'], cv=5, n_jobs=-1)

In [ ]:
#Entrenamos el modelo
gs.fit(dataa)

In [ ]:
print(f"Mejores hiperparámetros encontrados: {gs.best_params['rmse']}")


Mejores hiperparámetros encontrados: {'n_factors': 50, 'n_epochs': 30, 'lr_all': 0.002, 'reg_all': 0.04}


In [ ]:
# Con base a los mejores hiperparametros que encontramos, creamos el objeto SVD
model = SVD(n_factors=50, n_epochs=30, lr_all=0.002, reg_all=0.04)

# Entrenar el modelo en el conjunto de entrenamiento
model.fit(trainset)

El modelo se probo con el 100, 50, 25 y 10 porciento de los datos. Buscando la mejor accuracy. 

Nuestro mejor resultado fue con 25%, con una media de 0.9953 de RMSE

In [ ]:
# 100 % del dataset

# Hacer predicciones en el conjunto de prueba
predictions = model.test(testset)

# Calcular la precisión del modelo
accuracy.rmse(predictions)

RMSE: 1.0024


1.0023995069981524

In [ ]:
# 50 % del dataset

# Hacer predicciones en el conjunto de prueba
predictions = model.test(testset)

# Calcular la precisión del modelo
accuracy.rmse(predictions)

RMSE: 0.9964


0.996404607578528

In [ ]:
# 25 % del dataset prueba final

# Hacer predicciones en el conjunto de prueba
predictions = model.test(testset)

# Calcular la precisión del modelo
accuracy.rmse(predictions)

RMSE: 0.9829


0.982890444199184

In [ ]:
# 25 % del dataset prueba 2

results = cross_validate(model, dataa, measures=['RMSE'], cv=5, verbose=True)
print(results['test_rmse'])

Evaluating RMSE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9950  0.9947  0.9955  0.9946  0.9966  0.9953  0.0007  
Fit time          62.93   65.35   64.23   65.09   65.51   64.62   0.95    
Test time         11.62   17.59   12.03   11.64   11.75   12.93   2.33    
[0.99496964 0.99474768 0.99553255 0.9946437  0.99655732]


In [ ]:
# 10 % del dataset

# Hacer predicciones en el conjunto de prueba
predictions = model.test(testset)

# Calcular la precisión del modelo
accuracy.rmse(predictions)

RMSE: 1.0053


1.0053403625603339

In [ ]:
#Probamos que el modelo funcione de manera correcta
userId = 266225
movieId = 'ns643'
rating = model.predict(userId, movieId).est

In [ ]:
print(rating)

2.9054038105465723


In [51]:
#Exportamos nuestro modelo 
from surprise.dump import dump

dump("modelo_25.pkl", algo=model)